In [1]:
from PySide2 import QtCore
%gui qt

In [3]:
import mbuild as mb
import numpy as np
import fresnel
import fresnel.interact
import PIL

import utils
from cg_compound import CG_Compound
from draw_scene import visualize

/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/hoomd/meta.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [34]:
hexane = mb.load("CCCCCC", smiles=True)
hexane.visualize().show()

# convert to pybel mol
mol = hexane.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol.OBMol.PerceiveBondOrders()

hexane_sites = [("_A", "C[CH3]"), ("_B", "CC")]
cg_hexane = utils.coarse(mol, hexane_sites)

cg_hexane.visualize(show_atomistic=True, color_scheme={"_A": "blue", "_B": "orange"}).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<_A pos=( 0.1636, 0.0003, 0.0029), 0 bonds, id: 5082898768>" is element: "EP"
  atom, element))
/Users/jenny/miniconda3/envs/grits/lib/python3.7/site-packages/mbuild/compound.py:2443: UserWarning: Guessing that "<_B pos=( 0.3694, 0.1412, 0.0358), 0 bonds, id: 5082899024>" is element: "EP"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [27]:
scene = utils.draw_scene(cg_hexane, color="spring", show_atomistic=True)

In [28]:
view = fresnel.interact.SceneView(scene)
view.show()

In [29]:
output = fresnel.pathtrace(scene, light_samples=40, w=600, h=600)
filename = "hexane_scene.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [3]:
peek = mb.load("peek_opt.mol2")
#print(peek.n_particles)
#print(peek.n_bonds)
#peek.visualize()

In [4]:
scene = utils.draw_scene(peek)
scene.camera = fresnel.camera.orthographic(
    position=(-0.7705824971199036, -0.820938766002655, 0.3231777548789978), 
    look_at=(-0.8508099913597107, -0.0007099993526935577, 0.3265799880027771), 
    up=(0.9950225353240967, 0.09723380208015442, 0.021809743717312813), 
    height=1.7
)

In [5]:
fresnel.interact.SceneView(scene)

In [6]:
good_camera = scene.camera

In [7]:
output = fresnel.pathtrace(scene, light_samples=40, w=600, h=600)
filename = "peek_scene.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [8]:
# convert to pybel mol
mol = peek.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol.OBMol.PerceiveBondOrders()


cg_peek = utils.coarse(mol, [("_B","c1ccccc1")])

/Users/jenny/miniconda3/envs/fresnel/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')


In [9]:
cg_peek.add_bond((cg_peek[0],cg_peek[1]))
cg_peek.add_bond((cg_peek[1],cg_peek[2]))
cg_peek.visualize(show_atomistic=True).show()

/Users/jenny/miniconda3/envs/fresnel/lib/python3.7/site-packages/mbuild/compound.py:2415: UserWarning: Guessing that "<_B pos=(-0.8608,-0.0011, 0.8194), 0 bonds, id: 5089547880>" is element: "EP"
  atom, element))


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
cg_scene = utils.draw_scene(cg_peek, color="viridis", show_atomistic=True)
cg_scene.camera = good_camera

In [11]:
output = fresnel.pathtrace(cg_scene, light_samples=40, w=600, h=600)
filename = "cgpeek_scene1.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [12]:
cg_peek.remove(cg_peek[2])
cg_peek.remove(cg_peek[0])

In [13]:
cg_scene = utils.draw_scene(cg_peek, cg_scale=12, show_atomistic=True)
cg_scene.camera = good_camera

In [14]:
fresnel.interact.SceneView(cg_scene)

In [15]:
output = fresnel.pathtrace(cg_scene, light_samples=40, w=600, h=600)
filename = "cgpeek_scene2.png"

image = PIL.Image.fromarray(output[:], mode='RGBA')
image.save(filename, dpi=(300, 300))

In [ ]:
class PEEK(mb.Compound):
    def __init__(self):
        super(PEEK, self).__init__()
        mb.load("peek_opt.mol2", compound=self)
        # replace the fluorines with ports
        f_bonds = [ b for b in self.bonds() if b[0].name == "F" or b[1].name == "F" ]
        for pair in f_bonds:
            if pair[0].name == "F":
                f = pair[0]
                anchor = pair[1]
            else:
                f = pair[1]
                anchor = pair[0]
            orientation = f.pos - anchor.pos
            # half the length of the f_bond
            separation = np.linalg.norm(orientation)/2
            self.remove(f)
            if anchor.name == "O":
                self.add(mb.Port(anchor=anchor, orientation=orientation, separation=separation), label='up')
            else:
                self.add(mb.Port(anchor=anchor, orientation=orientation, separation=separation), label='down')

class H(mb.Compound):
    def __init__(self):
        super(H, self).__init__()
        self.add(mb.Particle(name='H', pos=[0,0,0]))
        self.add(mb.Port(anchor=self[0]), label='up')
        mb.translate(self['up'], [0, -0.154/2, 0]) 

In [ ]:
peek = PEEK()
peek.visualize(show_ports=True).show()

In [ ]:
class PEEK_Polymer(mb.Compound):
    def __init__(self, chain_length=1):
        super(PEEK_Polymer, self).__init__()
        last_monomer = PEEK()
        self.add(last_monomer)
        # add hydrogens on end caps
        end_h = H()
        mb.force_overlap(move_this=end_h, 
                 from_positions=end_h['up'], 
                 to_positions=last_monomer['down'])
        self.add(end_h)
        
        for i in range (chain_length-1):
            current_monomer = PEEK()

            mb.force_overlap(move_this=current_monomer, 
                             from_positions=current_monomer['down'], 
                             to_positions=last_monomer['up'])
            self.add(current_monomer)
            last_monomer=current_monomer
            
        # add end h
        end_h = H()
        mb.force_overlap(move_this=end_h, 
                         from_positions=end_h['up'], 
                         to_positions=last_monomer['up'])
        self.add(end_h)

In [ ]:
polymer = PEEK_Polymer(5)
polymer.visualize().show()

In [ ]:
# convert to pybel mol
mol = polymer.to_pybel()

# to_pybel imports all bonds as order=1, this will type the bond correctly
# if the structure is good
mol.OBMol.PerceiveBondOrders()


cg_peek = utils.coarse(mol, [("_B","O=C(c1ccccc1)"),("_A", "Oc1ccccc1")])


In [ ]:
cg_peek.visualize(show_atomistic=True)